In [1]:
import numpy as np
import pandas as pd

In [2]:
#reading the data from the different excel-files

start = 2013
stop = 2021

years = range(start, stop+1)
str_form = 'xlsx/elspot-prices_{}_hourly_eur.xlsx'

files = {}
for year in years:
    path = str_form.format(year)
    price_data = pd.read_excel(path)
    files[year] = price_data

In [3]:
#Checking out the different columns, the areas for which we have prices

''''
for year in files.keys():
    print('Year: ' + str(year))
    print(files[year].head())
'''

#Finding the intersection/columns for which we have prices
prices = None
first = True

for year in files.keys():
    cols = set(files[year].columns)
    if first:
        first = False
        prices = set(cols)
    prices = set.intersection(cols, prices)



In [4]:
#Deleting the columns which are not present in all the files

for year in files.keys():
    cols = files[year].columns
    for col in cols:
        if col not in prices:
            del files[year][col]


In [5]:
#Making a joint dataFrame

all_prices = None
first = True

for year in files.keys():
    if first:
        first = False
        all_prices = files[year]
    else:
        all_prices = pd.concat([all_prices, files[year]])


all_prices.head()

,Unnamed: 0,Hours,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,Oslo,Kr.sand,Bergen,Molde,Tr.heim,Tromsø,EE,LV,LT
0,2013-01-01,00 - 01,31.05,31.04,31.04,31.04,31.04,31.04,14.03,14.03,32.98,32.98,32.98,31.04,31.04,31.04,31.12,NaN,24.42
1,2013-01-01,01 - 02,30.47,27.51,27.51,27.51,27.51,27.51,11.06,11.06,32.97,32.97,32.97,30.81,30.81,30.81,30.61,NaN,23.62
2,2013-01-01,02 - 03,28.92,24.44,24.44,24.44,24.44,24.44,8.50,8.50,32.59,32.59,32.59,30.77,30.77,30.77,24.44,NaN,23.93
3,2013-01-01,03 - 04,27.88,21.81,21.81,21.81,21.81,21.81,0.10,0.10,31.53,31.53,31.53,30.71,30.71,30.71,21.81,NaN,23.85
4,2013-01-01,04 - 05,26.96,22.37,22.37,22.37,22.37,22.37,2.01,2.01,30.54,30.54,30.54,30.63,30.63,30.63,22.37,NaN,23.26


In [6]:
#Making the columns containing date and hours to a joint column in a date format

#print(type(all_prices['Unnamed: 0'].iloc[7])) - is of the type of pandas.timestamp, and must be altered
print(str(all_prices['Unnamed: 0'].iloc[1354])[:10] + ' ' + all_prices['Hours'].iloc[1354][:2] + ':' + '00')

2013-02-26 10:00


In [7]:
from datetime import datetime
#Format(?): format='%Y-%m-%d %H:%M'
#+ ' ' + all_prices['Hours'][:2] + ':00'

all_prices['datestring'] = all_prices['Unnamed: 0'].astype(str) + ' ' + all_prices['Hours'].apply(lambda x: x[:2])

all_prices['datetime'] = all_prices['datestring'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H'))

all_prices['datetime'].head(25)

0    2013-01-01 00:00:00
1    2013-01-01 01:00:00
2    2013-01-01 02:00:00
3    2013-01-01 03:00:00
4    2013-01-01 04:00:00
5    2013-01-01 05:00:00
6    2013-01-01 06:00:00
7    2013-01-01 07:00:00
8    2013-01-01 08:00:00
9    2013-01-01 09:00:00
10   2013-01-01 10:00:00
11   2013-01-01 11:00:00
12   2013-01-01 12:00:00
13   2013-01-01 13:00:00
14   2013-01-01 14:00:00
15   2013-01-01 15:00:00
16   2013-01-01 16:00:00
17   2013-01-01 17:00:00
18   2013-01-01 18:00:00
19   2013-01-01 19:00:00
20   2013-01-01 20:00:00
21   2013-01-01 21:00:00
22   2013-01-01 22:00:00
23   2013-01-01 23:00:00
24   2013-01-02 00:00:00
Name: datetime, dtype: datetime64[ns]

In [8]:
#Removing unvalid values, such as N/A for prices - these should be numpy float 64 and check for 

areas = ['SYS', 'SE1', 'SE2', 'SE3', 'SE4', 'FI', 'DK1',
       'DK2', 'Oslo', 'Kr.sand', 'Bergen', 'Molde', 'Tr.heim', 'Tromsø', 'EE', 'LV', 'LT']

all_prices.describe()

areas_with_missing_values = ['EE', 'LV', 'LT'] #These have approx. 59 to 63 000 values - suggested removed
#These are Estonia, Latvia and Lithuania, which are only connected to Sweden and Finland

for area in areas_with_missing_values:
    del all_prices[area]


In [9]:
all_prices.columns

Index(['Unnamed: 0', 'Hours', 'SYS', 'SE1', 'SE2', 'SE3', 'SE4', 'FI', 'DK1',
       'DK2', 'Oslo', 'Kr.sand', 'Bergen', 'Molde', 'Tr.heim', 'Tromsø',
       'datestring', 'datetime'],
      dtype='object')

In [10]:
all_prices[['SYS', 'SE1', 'SE2', 'SE3', 'SE4', 'FI', 'DK1',
       'DK2', 'Oslo', 'Kr.sand', 'Bergen', 'Molde', 'Tr.heim', 'Tromsø']].cov()

,SYS,SE1,SE2,SE3,SE4,FI,DK1,DK2,Oslo,Kr.sand,Bergen,Molde,Tr.heim,Tromsø
SYS,447.682697,261.403830,261.593668,495.034651,539.397735,496.730800,551.477111,564.315156,491.483675,485.349343,485.409594,257.016227,257.016227,239.259089
SE1,261.403830,260.955685,261.085518,287.027737,287.272494,286.615452,261.732514,280.806504,249.708561,240.892303,242.438459,237.607923,237.607923,214.807103
SE2,261.593668,261.085518,261.491222,287.457676,287.853177,287.031398,262.367874,281.429524,249.865281,241.050692,242.597480,237.999299,237.999299,214.736877
SE3,495.034651,287.027737,287.457676,675.611947,709.775083,672.318792,680.855800,726.878264,539.634907,526.956025,528.563860,258.803850,258.803850,236.423654
SE4,539.397735,287.272494,287.853177,709.775083,856.072317,717.804512,830.924673,884.632225,602.165642,593.785695,591.235237,257.819728,257.819728,229.025072
FI,496.730800,286.615452,287.031398,672.318792,717.804512,831.109117,682.455377,738.492181,525.577094,514.415024,515.253383,253.725534,253.725534,228.659271
DK1,551.477111,261.732514,262.367874,680.855800,830.924673,682.455377,1193.172807,942.888690,646.511242,648.373480,641.015159,234.205873,234.205873,203.067105
DK2,564.315156,280.806504,281.429524,726.878264,884.632225,738.492181,942.888690,1002.999741,641.562334,634.413723,630.637575,248.962057,248.962057,217.170410
Oslo,491.483675,249.708561,249.865281,539.634907,602.165642,525.577094,646.511242,641.562334,615.198763,603.684535,604.837956,250.731796,250.731796,233.616666
Kr.sand,485.349343,240.892303,241.050692,526.956025,593.785695,514.415024,648.373480,634.413723,603.684535,603.878736,600.062898,242.271136,242.271136,224.864982


In [11]:
#Finding all the dates, which can be used for the hydro reservoir levels

dates_df = all_prices['datetime']
dates_df

0      2013-01-01 00:00:00
1      2013-01-01 01:00:00
2      2013-01-01 02:00:00
3      2013-01-01 03:00:00
4      2013-01-01 04:00:00
               ...        
8756   2021-12-31 19:00:00
8757   2021-12-31 20:00:00
8758   2021-12-31 21:00:00
8759   2021-12-31 22:00:00
8760   2021-12-31 23:00:00
Name: datetime, Length: 78897, dtype: datetime64[ns]

In [12]:
dates_df.to_csv('dates.csv')

In [13]:
print(all_prices['datetime'].iloc[11242])

2014-04-14 09:00:00
